In [1]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ATIVIDADE PRÁTICA - PRÉ-PROCESSAMENTO E SELEÇÃO DE CARACTERÍSTICAS: ROC
---

##  1) Pré-processamento de características. Na aula de hoje, vimos os procedimentos de remoção de outliers e de normalização do espaço de características. Nesta atividade, vamos garantir que compreendemos o que está envolvido nestes processos. Para isso, vamos utilizar funções que estão disponíveis no material do Moodle e que realizam este pré-processamento: em python, preprocess.py; em matlab, remoutliers.m, normaliza.m e preselec.m.